In [1]:
import yfinance as yf

In [2]:
def getStock(tic,period):
    ticker = yf.Ticker(tic)
    df = ticker.history(period=period)
    return df.values[:,:4],df.index

In [4]:
stock = 'BTC-USD'
data,ts = getStock(stock,'1y')

In [5]:
print(len(data))

366


In [ ]:
from All_Functions_Master_File import rsi,adder

In [ ]:
my_data = rsi(data,14,3,4)

In [ ]:
my_data[-1,4]

In [ ]:
n=len(my_data)
print(n)
print(len(ts[-n:]))

In [ ]:
from darts import TimeSeries
import numpy as np
#series = TimeSeries.from_times_and_values(ts[-n:],my_data[:,4],fill_missing_dates=True, freq='D')
series = TimeSeries.from_values(np.float32(my_data[:,4]))


In [ ]:
train, val = series[:100], series[100:]

In [ ]:
from darts.models import NBEATSModel
model = NBEATSModel(input_chunk_length=14, output_chunk_length=5)



In [ ]:
model.fit(train)

In [ ]:
pred = model.predict(n=len(val),series=train)
#pred = model.predict(n=len(val))

In [ ]:
import matplotlib.pyplot as plt

series.plot()
pred.plot(label='forecast', low_quantile=0.05, high_quantile=0.95)
plt.legend()

In [ ]:
from darts.metrics import mase
err = mase(series,pred,train)
print(err)

In [ ]:
def OnePrediction(hist,model,future):
    hist_series = TimeSeries.from_values(np.float32(hist))
    pred = model.predict(n=future,series=hist_series)
    return pred.mean()[0]
    
    

In [ ]:
my_data = adder(my_data,20)

In [ ]:
def simple_rsi_signal(Data, rsi_col,threshold, buy, sell):
    high = 100 - threshold
    low = threshold
    Data[:,buy] = 0
    Data[:,sell] = 0
    for i in range(len(Data)):
        
        if Data[i, rsi_col] < low and i > 14:
            
            Data[i, buy] = 1
            
        elif Data[i, rsi_col] > high:
            
            Data[i, sell] = -1    
    return Data

In [ ]:
my_data[:,5:10] = 0
my_data = simple_rsi_signal(my_data,4,30,5,6)

In [ ]:
print(my_data[:,5])

In [ ]:
import Primal_Functions_Performance_Evaluation 

In [ ]:
import importlib
importlib.reload(Primal_Functions_Performance_Evaluation)


expected_cost = 0.01
lot = 1
investment = 100
my_data_ret = Primal_Functions_Performance_Evaluation.holding(my_data, 5, 6, 7, 8)


In [ ]:
my_data_eq = Primal_Functions_Performance_Evaluation.equity_curve(my_data_ret, 7, expected_cost, lot, investment)
Primal_Functions_Performance_Evaluation.performance(my_data_eq, 7, my_data, stock,expected_cost, lot, investment)

In [ ]:
plt.plot(my_data_eq[:, 3], linewidth = 1, label = stock)

In [ ]:
def predicted_rsi_signal(Data, rsi_col,threshold,input_chunk_length, buy, sell):
    high = 100 - threshold
    low = threshold
    Data[:,buy] = 0
    Data[:,sell] = 0
    for i in range(len(Data)):
        if Data[i, rsi_col] < low and i > input_chunk_length:
            u = OnePrediction(Data[i-input_chunk_length:i+1, rsi_col] , model, 5)
            if u > Data[i, rsi_col]*1.1 :
                Data[i, buy] = 1
            
        elif Data[i, rsi_col] > 70 and i > input_chunk_length:
            u = OnePrediction(Data[i-input_chunk_length:i+1, rsi_col] , model, 5)
            if u < Data[i, rsi_col]:
                Data[i, sell] = -1    
    return Data

In [ ]:
my_data[:,9:13] = 0
my_data = predicted_rsi_signal(my_data,4,40,14,9,10)

In [ ]:
print(my_data[:,9])

In [ ]:
my_data_ret = Primal_Functions_Performance_Evaluation.holding(my_data, 9, 10, 11, 12)

In [ ]:
my_data_eq = Primal_Functions_Performance_Evaluation.equity_curve(my_data_ret, 11, expected_cost, lot, investment)
Primal_Functions_Performance_Evaluation.performance(my_data_eq, 11, my_data, stock,expected_cost, lot, investment)

In [ ]:
import Primal_Functions_Signal_Chart
import Primal_Functions_Double_Panel_Plotting
upper_barrier    = 75
lower_barrier    = 25
Primal_Functions_Signal_Chart.signal_chart_ohlc_color(my_data, stock, 3, 6, 7, window = 500)
Primal_Functions_Double_Panel_Plotting.indicator_plot_double(my_data, 0, 1, 2, 3, 4, window = 250)
plt.axhline(y = upper_barrier, color = 'black', linewidth = 1, linestyle = '--')
plt.axhline(y = lower_barrier, color = 'black', linewidth = 1, linestyle = '--')    

plt.plot(my_data_eq[:, 3], linewidth = 1, label = stock)
plt.grid()
plt.legend()
plt.axhline(y = investment, color = 'black', linewidth = 1)

In [ ]:
plt.plot(my_data_eq[:, 3], linewidth = 1, label = stock)

In [ ]:
import pandas as pd
df = pd.read_csv('backtest_rsi_signals.csv')

In [ ]:
df['performance'] = df[['baseline','ML']].apply(lambda x: x[1] - x[0], axis=1)

In [ ]:
list = df.loc[(df.performance>0),'asset' ].values

In [ ]:
print(len(list))

In [ ]:
list = df.loc[(df.ML>0),'asset' ].values

In [ ]:
print(len(list))

In [ ]:
from datetime import datetime,timedelta

In [ ]:
yesterday = datetime.today() - timedelta(days=1)

In [ ]:
d = datetime.today() - yesterday

In [ ]:
d.days